In [1]:
from pygsp import graphs, filters, plotting
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from Plotters import *
from Nets import *
import matplotlib as mpl
import pandas as pd
from ipywidgets import *
from functions import *
from sklearn import preprocessing, metrics
import community
from LouvainClustering_fast import Clustering
from LouvainClustering_fast import norm_var_information
plt.rcParams['image.cmap'] = 'jet'
pl = Plotters()

Could not load cython functions
No module named '_cython_fast_funcs'


In [2]:
%load_ext cython

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
#------------------------------------------------------------------------
# Run All Cells
#------------------------------------------------------------------------


num_net     =    2
name        =   'toy_example'
attributes  =   ['income']

#------------------------------------------------------------------------
# Loading network
#------------------------------------------------------------------------
data        =   Nets(num_net,attributes[0]) 
net         =   data.net
N           =   net.order()
partition   =   nx.get_node_attributes(net,"block")

#------------------------------------------------------------------------
# Getting grund truth
#------------------------------------------------------------------------
y_true      =   [net.node[n]['anomaly'] for n in net.nodes()] 
true_nodes  =   [i for i in range(0,len(y_true)) if y_true[i]==1]

#------------------------------------------------------------------------
# Setting Markov times
#------------------------------------------------------------------------
time        =   10**np.linspace(0,5,2000)
time        =   np.concatenate([np.array([0]),time])

In [5]:
#------------------------------------------------------------------------
# Setting layout for plotting the network
#------------------------------------------------------------------------
G = graphs.Community(N=160, Nc=4, comm_sizes=[30, 70, 35, 25] , seed=42)
G.set_coordinates(kind='community2D')
corrd = G.coords
#------------------------------------------------------------------------

2019-09-05 10:19:48,611:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.514866859365871


In [6]:
#------------------------------------------------------------------------
# Defining matrix of node attributes
#------------------------------------------------------------------------

f_matrix = np.zeros((N,len(attributes)))
for i, att in enumerate(attributes):
    attribs =  nx.get_node_attributes(net,att)
    f_matrix[:,i] =  list(attribs.values())


f_scaled  =  preprocessing.MinMaxScaler().fit_transform(f_matrix)
f  = np.matrix(f_scaled[:,0]).reshape(N,1)

#--------------------------------------------------------------------
A     = nx.adjacency_matrix(net).toarray()
sigma = 0.08
W     = get_weigth_matrix_vectors(A,f_scaled,sigma)

G = graphs.Graph(W)

G.compute_fourier_basis() 
G.set_coordinates(corrd)

#------------------------------------------------------------------------
# Random walk components
#------------------------------------------------------------------------
v_ones      =  np.matrix(np.ones((N,1)))
degree_vect =  W.sum(axis=1)                        # strengths vector
D           =  np.matrix(np.diag(degree_vect))    
avg_d       =  (v_ones.T*D*v_ones)[0,0]/N           # average strength

# stationary distribution
pi          =  v_ones.A.T.reshape(N)/N
#------------------------------------------------------------------------


In [7]:
#------------------------------------------------------------------------
# Plotting the concentration of energy on the network, the clusters found by Louvain algorithm when 
# evaluating the Markov stability at the same time than the filtering process, and the bars of concentration + threshold of anomalousness
#------------------------------------------------------------------------

def plot_heat(t=0):
    
    #------------------------------------------------------------------------
    # Computing Fourier basis evaluated at t
    #------------------------------------------------------------------------
    tau = time[int(t)]
            
    kernel     =  np.exp(-tau/avg_d*(G.e/G.lmax)) 
    exp_sigma  =  np.diag(kernel)
    Ht         =  np.dot(np.dot(G.U,exp_sigma),G.U.T)
    #------------------------------------------------------------------------
    
    print("Markov time: ",tau/G.lmax);print("\n");
    #------------------------------------------------------------------------
    # Plot Concentration 
    #------------------------------------------------------------------------
    fig, axes  =  plt.subplots(1, 2); 
    concent    =  np.linalg.norm(Ht, axis=0, ord=2)
       
    m_concent  =  np.matrix(concent.reshape(N,1))
    print("Global smoothness of concentration: ", m_concent.T*G.L.todense()*m_concent)
    pl.visualize_graph_signal(G,concent,title='t='+str(tau), ax=axes[0]) 
    axes[1]    = plt.imshow(W,interpolation='None')
    #cbar     = plt.colorbar(ax=axes[1])
    plt.title('W')
    plt.grid(False)        
    plt.show()
   #------------------------------------------------------------------------

    #------------------------------------------------------------------------
    #  Find clusters with Louvain algorithm and plot them
    #------------------------------------------------------------------------
    
    clustering        =  Clustering(p1=pi, p2=pi, T=Ht)    
    clustering.find_louvain_clustering(rnd_seed=3)
    clust_labels      =  np.array(list(clustering.partition.node_to_cluster_dict.values()))
    
    interp_com        =  interpolate_comm2(net,clust_labels, true_nodes)
    print("Num of clusters: ",len(set(interp_com)))

    node_labels = dict(zip(net.nodes(), np.array(interp_com, dtype='int')))
    
    fig, axes = plt.subplots(1, 2);    
    pl.visualize_graph_signal(G,interp_com,ax=axes[0], node_labels=node_labels)
    axes[1]   = plt.imshow(Ht,interpolation='None')
    plt.title('T = g_t(L)')
    plt.grid(False)  
    plt.show() 
    #------------------------------------------------------------------------
    
    #------------------------------------------------------------------------
    # Plotting concentration
    #------------------------------------------------------------------------
    comm_concent = np.zeros((N,2))
    comm_concent[:,0] = interp_com
    comm_concent[:,1] = concent
    
    df_comm_concent   = pd.DataFrame(comm_concent, columns=['groups','concentration'])
    
    ax = plt.subplot(1, 1, 1)
    std_val = df_comm_concent['concentration'].std()

    df_comm_concent.plot(kind='bar', title='Node concentration', grid=False, y='concentration',rot=0, ax=ax, cmap='viridis', fontsize=8, legend=False)
    plt.hlines(df_comm_concent['concentration'].mean() + 2.0*std_val, xmin=-1, xmax=170, linestyles='dashed', alpha=1.0, color='blue')
    ax.set_facecolor((1.0, 1.0, 1.0))

    return (interp_com, concent)

       

In [8]:
mpl.rcParams['figure.figsize'] = 18, 6

a = widgets.Text(value='0', continuous_update=False)
res_interact = interactive(plot_heat, t=a)
display(res_interact)

# Try:
#t = 682
#t = 1020
#t = 1715

interactive(children=(Text(value='0', continuous_update=False, description='t'), Output()), _dom_classes=('wid…

In [9]:
#--------------------------------------------------------------------------
# Compute optimal partitions for each t. That will be used to compute the variation of information.
#--------------------------------------------------------------------------
''''
mat_voi    =  np.zeros((len(time),len(time)))

mat_parts  = []    
for t in range(len(time)):
    
    kernel     =  np.exp(-time[t]/avg_d*(G.e/G.lmax)) 
    exp_sigma  =  np.diag(kernel)
    Ht         =  np.dot(np.dot(G.U,exp_sigma),G.U.T)
    clustering        =  Clustering(p1=pi, p2=pi, T=Ht)  
    clustering.find_louvain_clustering(rnd_seed=3)
    mat_parts.append(np.array(list(clustering.partition.node_to_cluster_dict.values())))

mat_parts = np.array(mat_parts)
np.savetxt('variation_of_information/toy_example_partitions.csv', mat_parts.T, fmt='%d')
np.savetxt('variation_of_information/toy_example_time.csv', time/G.lmax, fmt='%.4f')
'''

"'\nmat_voi    =  np.zeros((len(time),len(time)))\n\nmat_parts  = []    \nfor t in range(len(time)):\n    \n    kernel     =  np.exp(-time[t]/avg_d*(G.e/G.lmax)) \n    exp_sigma  =  np.diag(kernel)\n    Ht         =  np.dot(np.dot(G.U,exp_sigma),G.U.T)\n    clustering        =  Clustering(p1=pi, p2=pi, T=Ht)  \n    clustering.find_louvain_clustering(rnd_seed=3)\n    mat_parts.append(np.array(list(clustering.partition.node_to_cluster_dict.values())))\n\nmat_parts = np.array(mat_parts)\nnp.savetxt('variation_of_information/toy_example_partitions.csv', mat_parts.T, fmt='%d')\nnp.savetxt('variation_of_information/toy_example_time.csv', time/G.lmax, fmt='%.4f')\n"